In this notebook we carry out short-term persistence analysis of fog. This is using Markov chains.

Sidenote on what a Markov chain is:


> A Markov chain or Markov process is a stochastic model describing a sequence of possible events in which the probability of each event depends only on the state attained in the previous event. Informally, this may be thought of as, "What happens next depends only on the state of affairs now."
(Wikipedia)


## 1. Import Packages & Data

**df** is weather observations as obtained from Dublin Airport.

See 01_Data_Prep for list of new variables and preprocessing steps undertaken.

In [ ]:
# data processing
import pandas as pd
import numpy as np
from collections import Counter
import missingno
from scipy import stats

# visualisations
import matplotlib.pyplot as plt
import seaborn as sns

# other
from tqdm import tqdm
import os
import sys
seed=42

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# importing data and helper functions from directories dependent on which is available

joseph_path = '/content/drive/My Drive/DS_Modules/CA4021 (Final Year Project)/' # Joseph
julita_path = '/content/drive/My Drive/CA4021 (Final Year Project)/' # Julita

if os.path.exists(joseph_path):
  print("Importing from DS_Modules/CA4021")
  sys.path.append(os.path.join(joseph_path, 'scripts'))
  path = joseph_path

elif os.path.exists(julita_path):
  print("Importing directly from CA4021 folder")
  sys.path.append(sys.path.append(os.path.join(julita_path, 'scripts')))
  path = julita_path

Importing from DS_Modules/CA4021


In [ ]:
# import helper functions from aux file (prevents too much function definitions in the notebook)
from aux_functions import missing_percentages, plot_dist_discrete, plot_dist_continuous, \
plot_vis_discrete, plot_vis_continuous, month_vplot

In [ ]:
# import the preprocessed data
df = pd.read_csv(os.path.join(path, 'data/me_data_clean.csv'))  # weather data
df.index=pd.to_datetime(df.date_time)
fog_df = pd.read_csv(os.path.join(path, 'data/fog_duration_data.csv'))  # weather data
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

df.shape

(96432, 57)

In [ ]:
df


,date_time,year,month,day,hour,date,dir,speed,vis,ww,w,wwa,wa,pchar,ptend,cbl,msl,drybulb,wetbulb,dewpt,vp,rh,clow,cmedium,chigh,nlc,ntot,hlc,nsig1,tsig1,hsig1,nsig2,tsig2,hsig2,nsig3,tsig3,hsig3,nsig4,tsig4,hsig4,ceiling,sog,dos,weather,duration,rainfall,sunshine,tabdir,tabspeed,pweather,dni,vis_hr1,target_hr1,fog_state,season,temp_dew_dist,rainfall12hma
date_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2011-01-01 00:00:00,2011-01-01 00:00:00,2011,1,1,0,01-Jan-2011 00:00:00,27,7,9000,10,22,NaN,NaN,5,0.1,1017.1,1027.8,5.5,4.6,3.3,7.8,86,5.0,NaN,NaN,7,7,22,7,6,22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0,0.0,0.0,0.0,26,6,0,0,9000.0,no fog,no fog,winter,2.2,NaN
2011-01-01 01:00:00,2011-01-01 01:00:00,2011,1,1,1,01-Jan-2011 01:00:00,28,6,9000,10,22,NaN,NaN,5,0.0,1017.1,1027.8,5.1,4.4,3.4,7.8,89,5.0,NaN,NaN,7,7,22,7,6,22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0,0.0,0.0,0.0,28,6,0,0,8000.0,no fog,no fog,winter,1.7,NaN
2011-01-01 02:00:00,2011-01-01 02:00:00,2011,1,1,2,01-Jan-2011 02:00:00,27,6,8000,10,22,NaN,NaN,8,0.2,1016.8,1027.5,5.3,4.0,2.1,7.1,80,5.0,NaN,NaN,7,7,22,7,6,22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0,0.0,0.0,0.0,27,7,0,0,8000.0,no fog,no fog,winter,3.2,NaN
2011-01-01 03:00:00,2011-01-01 03:00:00,2011,1,1,3,01-Jan-2011 03:00:00,25,7,8000,10,22,NaN,NaN,7,0.5,1016.6,1027.3,5.2,4.6,3.7,8.0,90,5.0,NaN,NaN,7,7,23,7,6,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0,0.0,0.0,0.0,27,7,0,0,8000.0,no fog,no fog,winter,1.5,NaN
2011-01-01 04:00:00,2011-01-01 04:00:00,2011,1,1,4,01-Jan-2011 04:00:00,28,7,8000,10,22,NaN,NaN,6,0.5,1016.6,1027.3,5.1,4.7,4.1,8.2,94,5.0,NaN,NaN,7,7,24,7,6,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0,0.0,0.0,0.0,27,7,0,0,9000.0,no fog,no fog,winter,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 19:00:00,2021-12-31 19:00:00,2021,12,31,19,31-Dec-2021 19:00:00,20,7,20000,2,11,NaN,NaN,5,0.0,1000.5,1010.7,13.2,11.3,9.5,11.9,78,5.0,NaN,NaN,6,6,15,1,7,15,6.0,6.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,0.0,0.0,0,0.0,0.0,0.0,20,9,0,0,20000.0,no fog,no fog,winter,3.7,-7.549517e-15
2021-12-31 20:00:00,2021-12-31 20:00:00,2021,12,31,20,31-Dec-2021 20:00:00,19,8,20000,2,11,NaN,NaN,8,0.2,1000.1,1010.3,13.5,11.4,9.3,11.8,76,6.0,0.0,2.0,2,5,15,1,7,15,5.0,0.0,220.0,0.0,0.0,0.0,0.0,0.0,0.0,220.0,0.0,0.0,0,0.0,0.0,0.0,19,10,0,0,20000.0,no fog,no fog,winter,4.2,-7.549517e-15
2021-12-31 21:00:00,2021-12-31 21:00:00,2021,12,31,21,31-Dec-2021 21:00:00,10,9,20000,2,11,NaN,NaN,8,0.7,999.5,1009.7,12.7,10.9,9.1,11.6,79,6.0,0.0,2.0,2,4,15,1,7,15,4.0,0.0,220.0,0.0,0.0,0.0,0.0,0.0,0.0,220.0,0.0,0.0,0,0.0,0.0,0.0,10,6,0,0,20000.0,no fog,no fog,winter,3.6,-7.549517e-15


## 2. Carry out analysis

### 2.1 1st order Markov chain (looking at only the last hour to make predictions)

In [ ]:
p_00 = 0
p_11 = 0
p_01 = 0
p_10 = 0
for i in range(len(df)):
  # we will make a confusion matrix of the current and future state of fog. At first we are looking at the last hour to make predictions.
  if i == 0:
    continue
  # case 0: no fog now
  if df.iloc[i,53] == 'no fog':
    # case 0 0 : no fog now and no fog earlier
    if df.iloc[i-1,53] == 'no fog':
      p_00 += 1
      
    elif df.iloc[i-1,53] == 'fog':
      # case 1 0 : no fog now but fog earlier
      p_10 += 1
  # case 1: fog now
  elif df.iloc[i,53] == 'fog':
    # case 1 1 : fog now and fog earlier
    if df.iloc[i-1,53] == 'fog':
      p_11 += 1
      
    elif df.iloc[i-1,53] == 'no fog':
      # case 0 1 : fog now but no fog earlier
      p_01 += 1


In [ ]:
print(p_00, p_01)
print(p_10, p_11)

95429 254
254 494


In [ ]:
x = p_00 + p_01
y = p_10 + p_11

In [ ]:
print('           No fog (current)      Fog (current)')
print('No fog ',p_00/x, p_01/x)
print('Fog    ',p_10/y, p_11/y)

print("\n")
print('P = 1/2 * (p_00 + p_11)')
print('P =  ', 0.5*(p_00/x + p_11/y))

           No fog (current)      Fog (current)
No fog  0.997345400959418 0.0026545990405819216
Fog     0.339572192513369 0.660427807486631


P = 1/2 * (p_00 + p_11)
P =   0.8288866042230245


###2.2 2nd order Markov chains (relying on 2h past observations to make predictions)

In [ ]:
p_000 = 0
p_010 = 0
p_110 = 0
p_100 = 0
p_111 = 0
p_011 = 0
p_001 = 0
p_101 = 0

for i in range(len(df)):
  # we will make a confusion matrix of the current and future state of fog. We will now look at the past two hours to make predictions for the present.
  if i in [0,1]:
    continue
  # case 0: no fog now
  if df.iloc[i,53] == 'no fog':
    # case 0 0 0 : no fog -2h, no fog -1h, no fog now
    if df.iloc[i-2,53] == 'no fog' and df.iloc[i-1,53] == 'no fog':
      p_000 += 1
    # case 0 1 0 : no fog -2h, fog -1h, no fog now
    if df.iloc[i-2,53] == 'no fog' and df.iloc[i-1,53] == 'fog':
      p_010 += 1
    # case 1 1 0: fog -2h, fog -1h, no fog now
    if df.iloc[i-2,53] == 'fog' and df.iloc[i-1,53] == 'fog':
      p_110 += 1
    # case 1 0 0 : fog -2h, no fog -1h, no fog now
    if df.iloc[i-2,53] == 'fog' and df.iloc[i-1,53] == 'no fog':
      p_100 += 1
  # case 1: fog now  
  elif df.iloc[i,53] == 'fog':
    # case 0 0 1: no fog -2h, no fog -1h, fog now:
    if df.iloc[i-2,53] == 'no fog' and df.iloc[i-1,53] == 'no fog':
      p_001 += 1
    # case 0 1 1 : no fog -2h, fog 1h, fog now
    if df.iloc[i-2,53] == 'no fog' and df.iloc[i-1,53] == 'fog':
      p_011 += 1
    # case 1 0 1 : fog -2h, no fog -1h, fog now
    if df.iloc[i-2,53] == 'fog' and df.iloc[i-1,53] == 'no fog':
      p_101 += 1
    # case 1 1 1 : fog -2h, fog -1h, fog now
    if df.iloc[i-2,53] == 'fog' and df.iloc[i-1,53] == 'fog':
      p_111 += 1

In [ ]:
print(p_000, p_001)
print(p_010, p_011)
print(p_100, p_101)
print(p_110, p_111)

95197 231
112 142
231 23
142 352


In [ ]:
print('           No fog (current)      Fog (current)')
print('0 0   ',p_000/(p_000+p_001) ,p_001/(p_000+p_001))
print('0 1   ',p_010/(p_010+p_011), p_011/(p_010+p_011))
print('1 0   ',p_100/(p_100+p_101), p_101/(p_100+p_101))
print('1 1   ',p_110/(p_110+p_111), p_111/(p_110+p_111))

print("\n")
print('P = 1/2 * (p_000 + p_111)')
print('P =  ', 0.5*(p_000/(p_000+p_001) + p_111/(p_110+p_111)))

           No fog (current)      Fog (current)
0 0    0.9975793268223163 0.002420673177683699
0 1    0.4409448818897638 0.5590551181102362
1 0    0.9094488188976378 0.09055118110236221
1 1    0.2874493927125506 0.7125506072874493


P = 1/2 * (p_000 + p_111)
P =   0.8550649670548829


We can see that the probability of fog onset is higher if fog was present in the past two hours. p_111 went up to 0.71, whereas our previous p_11 which took into consideration only one past hour had probability of 0.66

###2.3 3rd order Markov chains (relying on 3h past observations to make predictions)

In [ ]:

d = {}
for j in range(0,4):
  temp=""
  for i in range(len(df)):
      # parse each sequence of 4 time points, starting from jth point
    if j > 0 and i in range(0,j):
      # start from a different row every time (to get all observations)
      continue
    if df.iloc[i,53] == 'no fog':
      temp += "0"
      # when 4 time points have been parsed, update the dictionary and start the next 4
      if len(temp) == 4:
        if temp in d:
          d[temp] += 1
        else:
          d[temp] = 1
        temp = ""
    else:
      temp += "1"
      if len(temp) == 4:
        if temp in d.keys():
          d[temp] += 1
        else:
          d[temp] = 1
        temp = ""
    


0
{}
1
{'0000': 23748, '1111': 69, '1100': 37, '1110': 20, '0100': 25, '0011': 30, '1000': 62, '0111': 21, '0001': 49, '1001': 1, '1011': 5, '1101': 3, '0010': 23, '0101': 2, '0110': 10, '1010': 3}
2
{'0000': 47487, '1111': 129, '1100': 66, '1110': 50, '0100': 46, '0011': 63, '1000': 120, '0111': 46, '0001': 119, '1001': 5, '1011': 7, '1101': 4, '0010': 40, '0101': 7, '0110': 20, '1010': 6}
3
{'0000': 71231, '1111': 192, '1100': 104, '1110': 72, '0100': 64, '0011': 100, '1000': 169, '0111': 71, '0001': 172, '1001': 6, '1011': 11, '1101': 6, '0010': 77, '0101': 9, '0110': 30, '1010': 8}


In [ ]:
print(d['0000'], d['0001'])
print(d['0010'], d['0011'])
print(d['0100'], d['0101'])
print(d['1000'], d['1001'])
print(d['1100'], d['1101'])
print(d['0110'], d['0111'])
print(d['1010'], d['1011'])
print(d['1110'], d['1111'])

94974 222
102 129
100 12
222 9
131 11
46 96
10 13
96 256


In [ ]:
print('           No fog (current)      Fog (current)')
print('0 0 0  ',d['0000']/(d['0000']+d['0001']) ,d['0001']/(d['0000']+d['0001']))
print('0 0 1   ',d['0010']/(d['0010']+d['0011']), d['0011']/(d['0010']+d['0011']))
print('0 1 0   ',d['0100']/(d['0100']+d['0101']), d['0101']/(d['0100']+d['0101']))
print('1 0 0 ',d['1000']/(d['1000']+d['1001']), d['1001']/(d['1000']+d['1001']))
print('1 1 0  ',d['1100']/(d['1100']+d['1101']) ,d['1101']/(d['1100']+d['1101']))
print('0 1 1   ',d['0110']/(d['0110']+d['0111']), d['0111']/(d['0110']+d['0111']))
print('1 0 1   ',d['1010']/(d['1010']+d['1011']), d['1011']/(d['1010']+d['1011']))
print('1 1 1 ',d['1110']/(d['1110']+d['1111']), d['1111']/(d['1110']+d['1111']))

print("\n")
print('P = 1/2 * (p_0000 + p_1111)')
print('P =  ', 0.5*(d['0000']/(d['0000']+d['0001']) + d['1111']/(d['1110']+d['1111'])))

           No fog (current)      Fog (current)
0 0 0   0.9976679692424052 0.0023320307575948568
0 0 1    0.44155844155844154 0.5584415584415584
0 1 0    0.8928571428571429 0.10714285714285714
1 0 0  0.961038961038961 0.03896103896103896
1 1 0   0.9225352112676056 0.07746478873239436
0 1 1    0.323943661971831 0.676056338028169
1 0 1    0.43478260869565216 0.5652173913043478
1 1 1  0.2727272727272727 0.7272727272727273


P = 1/2 * (p_0000 + p_1111)
P =   0.8624703482575662


## 4th order Markov chains

In [ ]:

d = {}
for j in range(0,5):
  temp = ""
  for i in range(len(df)):
    if j > 0 and i in range(0,j):
      continue
    if df.iloc[i,53] == 'no fog':
      temp += "0"
      if len(temp) == 5:
        if temp in d:
          d[temp] += 1
        else:
          d[temp] = 1
        temp = ""
    else:
      temp += "1"
      if len(temp) == 5:
        if temp in d.keys():
          d[temp] += 1
        else:
          d[temp] = 1
        temp = ""
    


In [ ]:
print('           No fog (current)      Fog (current)')
print('0 0 0 0 ',d['00000']/sum([d['00000'], d['00001']]), d['00001']/sum([d['00000'], d['00001']]))
print('0 0 0 1 ',d['00010']/sum([d['00010'], d['00011']]), d['00011']/sum([d['00010'], d['00011']]))
print('0 0 1 0 ',d['00100']/sum([d['00100'], d['00101']]), d['00101']/sum([d['00100'], d['00101']]))
print('0 0 1 1 ',d['00110']/sum([d['00110'], d['00111']]), d['00111']/sum([d['00110'], d['00111']]))
print('0 1 0 0 ',d['01000']/sum([d['01000'], d['01001']]), d['01001']/sum([d['01000'], d['01001']]))
print('0 1 0 1 ',d['01010']/sum([d['01010'], d['01011']]), d['01011']/sum([d['01010'], d['01011']]))
print('1 0 0 0 ',d['10000']/sum([d['10000'], d['10001']]), d['10001']/sum([d['10000'], d['10001']]))
print('1 0 0 1 ',d['10010']/sum([d['10010'], d['10011']]), d['10011']/sum([d['10010'], d['10011']]))
print('1 1 0 0 ',d['11000']/sum([d['11000'], d['11001']]), d['11001']/sum([d['11000'], d['11001']]))
print('1 1 0 1 ',d['11010']/sum([d['11010'], d['11011']]), d['11011']/sum([d['11010'], d['11011']]))
print('0 1 1 0 ',d['01100']/sum([d['01100'], d['01101']]), d['01101']/sum([d['01100'], d['01101']])) 
print('0 1 1 1 ',d['01110']/sum([d['01110'], d['01111']]), d['01111']/sum([d['01110'], d['01111']]))
print('1 0 1 0 ',d['10100']/sum([d['10100'], d['10101']]), d['10101']/sum([d['10100'], d['10101']]))
print('1 0 1 1 ',d['10110']/sum([d['10110'], d['10111']]), d['10111']/sum([d['10110'], d['10111']]))
print('1 1 1 0 ',d['11100']/sum([d['11100'], d['11101']]), d['11101']/sum([d['11100'], d['11101']]))
print('1 1 1 1 ',d['11110']/sum([d['11110'], d['11111']]), d['11111']/sum([d['11110'], d['11111']]))

print("\n")
print('P = 1/2 * (p_00000 + p_11111)')
print('P =  ', 0.5*(d['00000']/(d['00000']+d['00001']) + d['11111']/(d['11110']+d['11111'])))

           No fog (current)      Fog (current)
0 0 0 0  0.9976835521674581 0.00231644783254188
0 0 0 1  0.45495495495495497 0.545045045045045
0 0 1 0  0.9117647058823529 0.08823529411764706
0 0 1 1  0.3178294573643411 0.6821705426356589
0 1 0 0  0.98 0.02
0 1 0 1  0.3333333333333333 0.6666666666666666
1 0 0 0  0.990990990990991 0.009009009009009009
1 0 0 1  0.1111111111111111 0.8888888888888888
1 1 0 0  0.9465648854961832 0.05343511450381679
1 1 0 1  0.5454545454545454 0.45454545454545453
0 1 1 0  0.9565217391304348 0.043478260869565216
0 1 1 1  0.3020833333333333 0.6979166666666666
1 0 1 0  0.7 0.3
1 0 1 1  0.38461538461538464 0.6153846153846154
1 1 1 0  0.90625 0.09375
1 1 1 1  0.26171875 0.73828125


P = 1/2 * (p_00000 + p_11111)
P =   0.867982401083729
